#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# BD Integrsas e integronas y mas de Shewanella

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [15]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display

def instalar_librerias(b=""):
    with output:
        librerias = [
            "import_ipynb",
            "nbconvert",
            "ipywidgets",  # Asegúrate de que ya esté instalado
            "biopython",
            "pandas",
            "numpy",
            "matplotlib",
            "tqdm",
            "odfpy",
            "requests",
            "gdown"
        ]

        barra_progreso = widgets.IntProgress(
            value=0,
            min=0,
            max=len(librerias),
            bar_style='info',
            orientation='horizontal'
        )

        etiqueta = widgets.Label(value='Iniciando instalación...')
        display(widgets.VBox([etiqueta, barra_progreso]))

        for libreria in librerias:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
            barra_progreso.value += 1

        etiqueta.value = 'Instalación completada.'

# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)
#instalar_librerias()

Output()

In [16]:
#@title 2) descargar cvs con id

import subprocess
from tqdm.notebook import tqdm

def descargar_archivos_desde_google_drive(ids_archivos):
    for id_archivo in tqdm(ids_archivos, desc="Descargando archivos"):
        url = f"https://drive.google.com/uc?id={id_archivo}"
        comando_descarga = ["gdown", "--id", id_archivo]
        proceso = subprocess.Popen(comando_descarga, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        stdout, stderr = proceso.communicate()

        if proceso.returncode != 0:
            print(f"Error al descargar el archivo con ID {id_archivo}: {stderr.decode().strip()}")
        else:
            print(f"Archivo con ID {id_archivo} descargado exitosamente.")

#falla excel "1B4smxRJe77roNY-b3u7lMjdKd7vM_dREp3_3O_UdtmU"   # integrones_shewanella
# Lista de IDs de archivos de Google Drive
ids_archivos = [
    "1mkYUcDlFZLVSc2tTJMgOV7xA4PvegO4_",  # Integrasas_de_Integron_Shew.csv
    "16RdgLIn4GANuLmoayAI20QEt086SqBzh",  # Integrasas_de_Integron.csv
    "1fHxLBiSnpXPXULMxnuGumJFgM46ZVlIh",  # Integrasas_de_Integron.ods
    "1M_JRAG74APAUTVXiZ-z9iPtXjQ2Md__3"   # integrones_shewanella
]

# Llamar a la función
#descargar_archivos_desde_google_drive(ids_archivos)

In [25]:
import pandas as pd

def leer_y_mostrar_archivo(ruta_archivo, tipo_archivo='csv', separador='\t'):
    if tipo_archivo == 'csv':
        df = pd.read_csv(ruta_archivo, sep=separador)
    elif tipo_archivo == 'ods':
        df = pd.read_excel(ruta_archivo, engine='odf')
    else:
        raise ValueError("Tipo de archivo no soportado")

    print(df)
    print("Columnas:", df.columns.tolist())
    print("\n #################################################### \n")

# Ejemplos de uso:
#leer_y_mostrar_archivo('Integrasas_de_Integron.csv', 'csv')
leer_y_mostrar_archivo('Integrasas_de_Integron_Shew.csv', 'csv')
#leer_y_mostrar_archivo('Integrasas_de_Integron.ods', 'ods')
leer_y_mostrar_archivo('integrones_shewanella.csv', 'csv')

        PROTEIN_ID Integrase type                              Organism
0   WP_000845048.1          IntI1  Shewanella xiamenensis strain HD6416
1      AVT49498.1           IntI1                Shewanella baltica CW2
2      AYV11497.1           IntI1             Shewanella algae KC-Na-R1
3      QHD51725.1           IntI1        Shewanella algae 18064-CSB-B-B
4      AVV61730.1           IntI1        Shewanella sp. strain 12P2G11 
5      QQK62381.1           IntI1                    Shewanella sp. LC6
6      ODR83682.1           IntI1            Shewanella xiamenensis T17
7       TPE62986.1          IntI1                    Shewanella sp. LC2
8      TVK96596.1           IntI1                 Shewanella algae CLS1
9      PKH30948.1           IntI1                   Shewanella sp. ALD9
10     AVV61734.1           IntI1              Shewanella baltica 30PF3
Columnas: ['PROTEIN_ID', 'Integrase type', 'Organism']

 #################################################### 

                        

In [18]:
import re
def clean_protein_ids(proteins):
    clean_proteins = []
    for protein in proteins:
        clean_protein = re.sub(r'[^A-Za-z0-9.]', '', protein)
        clean_proteins.append(clean_protein)
    return clean_proteins

In [27]:
def ajustar_formato_ids(ids):
    ids_ajustados = []
    for id in ids:
        # Insertar un guion después del tercer carácter
        id_con_guion = id[:2] + '_' + id[2:]
        # Agregar ".1" al final
        id_final = id_con_guion + ".1"
        ids_ajustados.append(id_final)
    return ids_ajustados

In [30]:
#@title 2) descargar y guardar_secuencias_nucleotidicas

def descargar_y_guardar_secuencias_nucleotidicas0(ncbi_ids, lista_name=""):
    from Bio import Entrez, SeqIO
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq
    from urllib.error import HTTPError
    Entrez.email = "tu_email@example.com"  # Tu email real aquí
    secuencias = []

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="nucleotide", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            print("Se ha descargado: ",fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")
    print()
    return secuencias

In [31]:
#@title 2) descargar y guardar_secuencias_nucleotidicas

def descargar_y_guardar_secuencias_nucleotidicas(ncbi_ids, lista_name=""):
    from Bio import Entrez, SeqIO
    from urllib.error import HTTPError
    Entrez.email = "tu_email@example.com"  # Tu email real aquí
    secuencias = []
    nombres_archivos = []  # Lista para almacenar los nombres de los archivos

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="nucleotide", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            nombres_archivos.append(fasta_filename)  # Añadir el nombre del archivo a la lista
            print("Se ha descargado: ", fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")

    print()
    if lista_name:
        globals()[lista_name] = nombres_archivos  # Guardar la lista en una variable global con el nombre dado

    return secuencias, nombres_archivos  # Retornar las secuencias y los nombres de los archivos

In [33]:
#@title 2) descargar y guardar_secuencias_proteinas

# Función para descargar secuencias de proteínas desde NCBI y guardarlas en archivos FASTA
def descargar_y_guardar_secuencias_proteinas0(ncbi_ids):
    import pandas as pd
    from Bio import Entrez, SeqIO
    from Bio.Entrez import HTTPError
    Entrez.email = "tu_email@example.com"  # Sustituye esto con tu email real
    secuencias = []

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="protein", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            print("Se ha descargado: ",fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")
    print()
    return secuencias

In [34]:
#@title 2) descargar y guardar_secuencias_proteinas

# Función para descargar secuencias de proteínas desde NCBI y guardarlas en archivos FASTA
def descargar_y_guardar_secuencias_proteinas(ncbi_ids, lista_name=""):
    from Bio import Entrez, SeqIO
    from Bio.Entrez import HTTPError
    Entrez.email = "tu_email@example.com"  # Sustituye esto con tu email real
    secuencias = []
    nombres_archivos = []  # Lista para almacenar los nombres de los archivos

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="protein", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            nombres_archivos.append(fasta_filename)  # Añadir el nombre del archivo a la lista
            print("Se ha descargado: ", fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")

    print()
    if lista_name:
        globals()[lista_name] = nombres_archivos  # Guardar la lista en una variable global con el nombre dado

    return secuencias, nombres_archivos  # Retornar las secuencias y los nombres de los archivos


In [35]:
def main():
    # Funciones auxiliares previamente definidas
    instalar_librerias()
    import pandas as pd
    descargar_archivos_desde_google_drive(ids_archivos)

    # Descargar y analizar secuencias nucleotídicas
    df_nucleotidicas = pd.read_csv('Integrasas_de_Integron.csv', sep='\t')
    secuencias_nucleotidicas = descargar_y_guardar_secuencias_nucleotidicas(df_nucleotidicas['# GenBank '].tolist(), "integrasas_de_integron")

    # Descargar y analizar secuencias de proteínas (CSV)
    df_proteinas_csv = pd.read_csv('Integrasas_de_Integron_Shew.csv', sep='\t')
    protein_ids_csv = df_proteinas_csv['PROTEIN_ID'].tolist()
    clean_ids_csv = clean_protein_ids(protein_ids_csv)
    secuencias_proteinas_csv = descargar_y_guardar_secuencias_proteinas(clean_ids_csv, "integrasas_de_integron_shew")

    # Descargar y analizar secuencias de proteínas (CSV)
    df_proteinas2_csv = pd.read_csv('integrones_shewanella.csv', sep=',')
    protein_ids2_csv = df_proteinas2_csv['NCBI_ID'].tolist()
    clean_ids2_csv = clean_protein_ids(protein_ids2_csv)
    ids_ajustados = ajustar_formato_ids(clean_ids2_csv)
    secuencias_proteinas_csv = descargar_y_guardar_secuencias_proteinas(ids_ajustados, "integrones_shewanella")

    # Descargar y analizar secuencias de proteínas (ODS)
    file_path_ods = 'Integrasas_de_Integron.ods'
    df_proteinas_ods = pd.read_excel(file_path_ods, engine='odf')
    protein_ids_ods = df_proteinas_ods['ID protein'].tolist()
    clean_ids_ods = clean_protein_ids(protein_ids_ods)
    secuencias_proteinas_ods = descargar_y_guardar_secuencias_proteinas(clean_ids_ods, "integrasas_de_integron_ods")
# Ejecutar la función main
#main()

Descargando archivos:   0%|          | 0/4 [00:00<?, ?it/s]

Archivo con ID 1mkYUcDlFZLVSc2tTJMgOV7xA4PvegO4_ descargado exitosamente.
Archivo con ID 16RdgLIn4GANuLmoayAI20QEt086SqBzh descargado exitosamente.
Archivo con ID 1fHxLBiSnpXPXULMxnuGumJFgM46ZVlIh descargado exitosamente.
Archivo con ID 1M_JRAG74APAUTVXiZ-z9iPtXjQ2Md__3 descargado exitosamente.
Se ha descargado:  U12441.2.fasta
Se ha descargado:  AY509004.fasta
Se ha descargado:  AY259085.1.fasta
Se ha descargado:   AY183453.1.fasta
Se ha descargado:  EU780012.1.fasta
Se ha descargado:  AF416297.1.fasta
Se ha descargado:  AY219651.1.fasta

Error al descargar la secuencia para el ID WP000845048.1: HTTP Error 400: Bad Request
Se ha descargado:  AVT49498.1.fasta
Se ha descargado:  AYV11497.1.fasta
Se ha descargado:  QHD51725.1.fasta
Se ha descargado:  AVV61730.1.fasta
Se ha descargado:  QQK62381.1.fasta
Se ha descargado:  ODR83682.1.fasta
Se ha descargado:  TPE62986.1.fasta
Se ha descargado:  TVK96596.1.fasta
Se ha descargado:  PKH30948.1.fasta
Se ha descargado:  AVV61734.1.fasta

Se ha d


# teoria, codigo, pruebas, etc.

notebook1:

var = 5

def greeting():
  print("Hola!")


main:

import notebook1
print(notebook1.var)
notebook1.greeting()


----------------------------
Excelente pregunta.

Para visualizar un gráfico Matplotlib desde otro notebook que importa la función, hay un par de opciones:

1. Devuelve el objeto Figure desde la función:

```python
# notebook1.py
import matplotlib.pyplot as plt

def grafica():
   fig = plt.figure()
   plt.plot([1,2,3])
   return fig
```

```python
# notebook2.py

import notebook1

fig = notebook1.grafica()
fig # Se muestra el gráfico
```

2. Utiliza `matplotlib inline`:

```python
# notebook1.py

import matplotlib.pyplot as plt
plt.switch_backend('inline')

def grafica():
   plt.figure()
   plt.plot([1,2,3])
   plt.show()   
```

```python
# notebook2.py

import notebook1
notebook1.grafica() # Se muestra el gráfico
```

La opción 2 es más simple, `inline` hace que la imagen se muestre automáticamente.

Espero que esto te ayude a compartir visualizaciones entre notebooks de Colab. ¡Cualquier otra duda me comentas!
